In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Input, Activation, Dense
from tensorflow.python.keras.optimizers import SGD
from tensorflow.python.keras.utils.np_utils import to_categorical

In [2]:
# Pandas read CSV
sf_train = pd.read_csv('data/enemy_stats_training.csv')

In [3]:
# Correlation matrix for target
corr_matrix = sf_train.corr()

# Show how corelation matrix table
print(corr_matrix)

               Type    Levels        HP        MP  Strength     Magic  \
Type       1.000000  0.089835  0.140933 -0.250142  0.056565  0.051114   
Levels     0.089835  1.000000  0.610090  0.465260  0.615551  0.617927   
HP         0.140933  0.610090  1.000000  0.388518  0.459872  0.444550   
MP        -0.250142  0.465260  0.388518  1.000000  0.304559  0.247966   
Strength   0.056565  0.615551  0.459872  0.304559  1.000000  0.432294   
Magic      0.051114  0.617927  0.444550  0.247966  0.432294  1.000000   
Endurance -0.290492  0.408901  0.269480  0.294254  0.328022  0.394863   
Speed      0.043601  0.627556  0.373774  0.351847  0.384895  0.447145   
Luck       0.076972  0.604091  0.355588  0.245842  0.348243  0.268177   

           Endurance     Speed      Luck  
Type       -0.290492  0.043601  0.076972  
Levels      0.408901  0.627556  0.604091  
HP          0.269480  0.373774  0.355588  
MP          0.294254  0.351847  0.245842  
Strength    0.328022  0.384895  0.348243  
Magic      

In [4]:
# Show corelation matrix affect toward type
print(corr_matrix['Type'])

Type         1.000000
Levels       0.089835
HP           0.140933
MP          -0.250142
Strength     0.056565
Magic        0.051114
Endurance   -0.290492
Speed        0.043601
Luck         0.076972
Name: Type, dtype: float64


In [5]:
# Drop unnecessary columns
sf_train.drop(sf_train.columns[[2, 5, 6, 8, 9]], axis=1, inplace=True)

# Show current columns after drop the unnecessary
print(sf_train)

          Name  Type   HP   MP  Endurance
0      Enemy 1     0   42   45          2
1      Enemy 2     0   84   21          2
2      Enemy 3     2   43   51          6
3      Enemy 4     1   44  231          6
4      Enemy 5     2   40   40          6
..         ...   ...  ...  ...        ...
275  Enemy 276     2  257  293         11
276  Enemy 277     3  295   79          4
277  Enemy 278     4  272   68          2
278  Enemy 279     4   94   81         22
279  Enemy 280     0  182  289         15

[280 rows x 5 columns]


In [6]:
# Pandas read Validation CSV
sf_val = pd.read_csv('data/enemy_stats_validation.csv')

# Drop unnecessary columns
sf_val.drop(sf_val.columns[[2, 5, 6, 8, 9]], axis=1, inplace=True)

# Show current columns after drop the unnecessary
print(sf_val.head())

        Name  Type   HP   MP  Endurance
0  Enemy 281     2   91   92         11
1  Enemy 282     0  245   33         55
2  Enemy 283     2  246  249          5
3  Enemy 284     1  271  414         26
4  Enemy 285     4  136   81         25


In [7]:
# Get Pandas array value (Convert to NumPy array)
train_data = sf_train.values
val_data = sf_val.values

In [8]:
# Use columns 2 to last as Input
train_x = train_data[:,2:]
val_x = val_data[:,2:]

In [9]:
train_x.shape

(280, 3)

In [10]:
val_x.shape

(120, 3)

In [11]:
# Use columns 1 as output/target (one-hot encoding)
train_y = to_categorical(train_data[:,1])
val_y = to_categorical(val_data[:,1])

In [12]:
# Create network
inputs = Input(shape=(3,))
h_layer = Dense(6, activation='sigmoid')(inputs)

# Softmax activation for multiclass classification
outputs = Dense(5, activation='softmax')(h_layer)

In [13]:
model = Model(inputs=inputs, outputs=outputs)

# Optimizer / update rule
sgd = SGD(lr=0.001)

# Compile the model with cross entropy loss
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
# Train the model and use validation data
model.fit(train_x, train_y, batch_size=16, epochs=10000, verbose=1, validation_data=(val_x, val_y))
model.save_weights('enemy_char_classification.h5')

Train on 280 samples, validate on 120 samples
Epoch 1/10000
280/280 [==============================] - 0s 605us/sample - loss: 1.6180 - acc: 0.2571 - val_loss: 1.5777 - val_acc: 0.2583
Epoch 2/10000
280/280 [==============================] - 0s 117us/sample - loss: 1.6171 - acc: 0.2536 - val_loss: 1.5774 - val_acc: 0.2583
Epoch 3/10000
280/280 [==============================] - 0s 115us/sample - loss: 1.6162 - acc: 0.2536 - val_loss: 1.5772 - val_acc: 0.2583
Epoch 4/10000
280/280 [==============================] - 0s 132us/sample - loss: 1.6154 - acc: 0.2536 - val_loss: 1.5768 - val_acc: 0.2583
Epoch 5/10000
280/280 [==============================] - 0s 127us/sample - loss: 1.6145 - acc: 0.2536 - val_loss: 1.5766 - val_acc: 0.2583
Epoch 6/10000
280/280 [==============================] - 0s 112us/sample - loss: 1.6136 - acc: 0.2536 - val_loss: 1.5762 - val_acc: 0.2583
Epoch 7/10000
280/280 [==============================] - 0s 119us/sample - loss: 1.6127 - acc: 0.2536 - val_loss: 1.5758

280/280 [==============================] - 0s 97us/sample - loss: 1.5914 - acc: 0.2536 - val_loss: 1.5627 - val_acc: 0.2583
Epoch 60/10000
280/280 [==============================] - 0s 96us/sample - loss: 1.5912 - acc: 0.2500 - val_loss: 1.5626 - val_acc: 0.2583
Epoch 61/10000
280/280 [==============================] - 0s 94us/sample - loss: 1.5908 - acc: 0.2500 - val_loss: 1.5623 - val_acc: 0.2583
Epoch 62/10000
280/280 [==============================] - 0s 100us/sample - loss: 1.5906 - acc: 0.2500 - val_loss: 1.5622 - val_acc: 0.2583
Epoch 63/10000
280/280 [==============================] - 0s 97us/sample - loss: 1.5903 - acc: 0.2500 - val_loss: 1.5621 - val_acc: 0.2583
Epoch 64/10000
280/280 [==============================] - 0s 100us/sample - loss: 1.5900 - acc: 0.2500 - val_loss: 1.5619 - val_acc: 0.2583
Epoch 65/10000
280/280 [==============================] - 0s 100us/sample - loss: 1.5897 - acc: 0.2500 - val_loss: 1.5618 - val_acc: 0.2583
Epoch 66/10000
280/280 [===============

280/280 [==============================] - 0s 96us/sample - loss: 1.5769 - acc: 0.2536 - val_loss: 1.5531 - val_acc: 0.2583
Epoch 118/10000
280/280 [==============================] - 0s 101us/sample - loss: 1.5767 - acc: 0.2536 - val_loss: 1.5529 - val_acc: 0.2583
Epoch 119/10000
280/280 [==============================] - 0s 96us/sample - loss: 1.5765 - acc: 0.2536 - val_loss: 1.5528 - val_acc: 0.2583
Epoch 120/10000
280/280 [==============================] - 0s 100us/sample - loss: 1.5763 - acc: 0.2536 - val_loss: 1.5526 - val_acc: 0.2583
Epoch 121/10000
280/280 [==============================] - 0s 98us/sample - loss: 1.5760 - acc: 0.2536 - val_loss: 1.5525 - val_acc: 0.2583
Epoch 122/10000
280/280 [==============================] - 0s 99us/sample - loss: 1.5757 - acc: 0.2536 - val_loss: 1.5523 - val_acc: 0.2583
Epoch 123/10000
280/280 [==============================] - 0s 100us/sample - loss: 1.5755 - acc: 0.2536 - val_loss: 1.5521 - val_acc: 0.2583
Epoch 124/10000
280/280 [========

280/280 [==============================] - 0s 101us/sample - loss: 1.5646 - acc: 0.2571 - val_loss: 1.5441 - val_acc: 0.2583
Epoch 176/10000
280/280 [==============================] - 0s 97us/sample - loss: 1.5643 - acc: 0.2571 - val_loss: 1.5439 - val_acc: 0.2583
Epoch 177/10000
280/280 [==============================] - 0s 95us/sample - loss: 1.5641 - acc: 0.2571 - val_loss: 1.5438 - val_acc: 0.2583
Epoch 178/10000
280/280 [==============================] - 0s 97us/sample - loss: 1.5640 - acc: 0.2571 - val_loss: 1.5436 - val_acc: 0.2583
Epoch 179/10000
280/280 [==============================] - 0s 100us/sample - loss: 1.5637 - acc: 0.2571 - val_loss: 1.5434 - val_acc: 0.2583
Epoch 180/10000
280/280 [==============================] - 0s 99us/sample - loss: 1.5635 - acc: 0.2571 - val_loss: 1.5433 - val_acc: 0.2583
Epoch 181/10000
280/280 [==============================] - 0s 100us/sample - loss: 1.5634 - acc: 0.2571 - val_loss: 1.5430 - val_acc: 0.2583
Epoch 182/10000
280/280 [========

280/280 [==============================] - 0s 100us/sample - loss: 1.5528 - acc: 0.3393 - val_loss: 1.5331 - val_acc: 0.3667
Epoch 234/10000
280/280 [==============================] - 0s 97us/sample - loss: 1.5525 - acc: 0.3393 - val_loss: 1.5330 - val_acc: 0.3667
Epoch 235/10000
280/280 [==============================] - 0s 100us/sample - loss: 1.5523 - acc: 0.3357 - val_loss: 1.5328 - val_acc: 0.3667
Epoch 236/10000
280/280 [==============================] - 0s 106us/sample - loss: 1.5521 - acc: 0.3357 - val_loss: 1.5326 - val_acc: 0.3667
Epoch 237/10000
280/280 [==============================] - 0s 97us/sample - loss: 1.5519 - acc: 0.3357 - val_loss: 1.5324 - val_acc: 0.3667
Epoch 238/10000
280/280 [==============================] - 0s 93us/sample - loss: 1.5517 - acc: 0.3393 - val_loss: 1.5321 - val_acc: 0.3667
Epoch 239/10000
280/280 [==============================] - 0s 97us/sample - loss: 1.5514 - acc: 0.3393 - val_loss: 1.5320 - val_acc: 0.3667
Epoch 240/10000
280/280 [========

...

Epoch 9726/10000
280/280 [==============================] - 0s 92us/sample - loss: 1.0291 - acc: 0.4214 - val_loss: 1.2248 - val_acc: 0.3583
Epoch 9727/10000
280/280 [==============================] - 0s 103us/sample - loss: 1.0289 - acc: 0.4214 - val_loss: 1.1816 - val_acc: 0.3750
Epoch 9728/10000
280/280 [==============================] - 0s 89us/sample - loss: 1.0346 - acc: 0.4143 - val_loss: 1.3318 - val_acc: 0.3500
Epoch 9729/10000
280/280 [==============================] - 0s 93us/sample - loss: 1.0285 - acc: 0.4250 - val_loss: 1.1698 - val_acc: 0.3750
Epoch 9730/10000
280/280 [==============================] - 0s 92us/sample - loss: 1.0354 - acc: 0.4214 - val_loss: 1.2084 - val_acc: 0.3583
Epoch 9731/10000
280/280 [==============================] - 0s 89us/sample - loss: 1.0265 - acc: 0.4214 - val_loss: 1.1728 - val_acc: 0.3833
Epoch 9732/10000
280/280 [==============================] - 0s 95us/sample - loss: 1.0245 - acc: 0.4357 - val_loss: 1.3638 - val_acc: 0.3500
Epoch 9733/1

280/280 [==============================] - 0s 93us/sample - loss: 1.0441 - acc: 0.4214 - val_loss: 1.1768 - val_acc: 0.3667
Epoch 9785/10000
280/280 [==============================] - 0s 93us/sample - loss: 1.0271 - acc: 0.4179 - val_loss: 1.2090 - val_acc: 0.3583
Epoch 9786/10000
280/280 [==============================] - 0s 93us/sample - loss: 1.0311 - acc: 0.4179 - val_loss: 1.1821 - val_acc: 0.3750
Epoch 9787/10000
280/280 [==============================] - 0s 90us/sample - loss: 1.0347 - acc: 0.4179 - val_loss: 1.2377 - val_acc: 0.3583
Epoch 9788/10000
280/280 [==============================] - 0s 89us/sample - loss: 1.0319 - acc: 0.4286 - val_loss: 1.1700 - val_acc: 0.3833
Epoch 9789/10000
280/280 [==============================] - 0s 89us/sample - loss: 1.0291 - acc: 0.4143 - val_loss: 1.1856 - val_acc: 0.3750
Epoch 9790/10000
280/280 [==============================] - 0s 89us/sample - loss: 1.0303 - acc: 0.4250 - val_loss: 1.1870 - val_acc: 0.3667
Epoch 9791/10000
280/280 [====

280/280 [==============================] - 0s 108us/sample - loss: 1.0334 - acc: 0.4321 - val_loss: 1.1746 - val_acc: 0.3917
Epoch 9843/10000
280/280 [==============================] - 0s 107us/sample - loss: 1.0287 - acc: 0.4214 - val_loss: 1.1940 - val_acc: 0.3750
Epoch 9844/10000
280/280 [==============================] - 0s 108us/sample - loss: 1.0273 - acc: 0.4179 - val_loss: 1.2724 - val_acc: 0.3500
Epoch 9845/10000
280/280 [==============================] - 0s 110us/sample - loss: 1.0273 - acc: 0.4214 - val_loss: 1.2296 - val_acc: 0.3583
Epoch 9846/10000
280/280 [==============================] - 0s 110us/sample - loss: 1.0299 - acc: 0.4250 - val_loss: 1.2761 - val_acc: 0.3500
Epoch 9847/10000
280/280 [==============================] - 0s 110us/sample - loss: 1.0287 - acc: 0.4179 - val_loss: 1.2859 - val_acc: 0.3500
Epoch 9848/10000
280/280 [==============================] - 0s 125us/sample - loss: 1.0248 - acc: 0.4250 - val_loss: 1.1694 - val_acc: 0.4000
Epoch 9849/10000
280/28

Epoch 9900/10000
280/280 [==============================] - 0s 103us/sample - loss: 1.0307 - acc: 0.4143 - val_loss: 1.2361 - val_acc: 0.3583
Epoch 9901/10000
280/280 [==============================] - 0s 89us/sample - loss: 1.0281 - acc: 0.4143 - val_loss: 1.1718 - val_acc: 0.3833
Epoch 9902/10000
280/280 [==============================] - 0s 100us/sample - loss: 1.0244 - acc: 0.4179 - val_loss: 1.2953 - val_acc: 0.3500
Epoch 9903/10000
280/280 [==============================] - 0s 96us/sample - loss: 1.0295 - acc: 0.4214 - val_loss: 1.2192 - val_acc: 0.3583
Epoch 9904/10000
280/280 [==============================] - 0s 96us/sample - loss: 1.0262 - acc: 0.4250 - val_loss: 1.2703 - val_acc: 0.3500
Epoch 9905/10000
280/280 [==============================] - 0s 97us/sample - loss: 1.0264 - acc: 0.4250 - val_loss: 1.1754 - val_acc: 0.3833
Epoch 9906/10000
280/280 [==============================] - 0s 100us/sample - loss: 1.0303 - acc: 0.4179 - val_loss: 1.2155 - val_acc: 0.3583
Epoch 9907

Epoch 9958/10000
280/280 [==============================] - 0s 101us/sample - loss: 1.0259 - acc: 0.4214 - val_loss: 1.3386 - val_acc: 0.3500
Epoch 9959/10000
280/280 [==============================] - 0s 101us/sample - loss: 1.0332 - acc: 0.4214 - val_loss: 1.2792 - val_acc: 0.3500
Epoch 9960/10000
280/280 [==============================] - 0s 103us/sample - loss: 1.0280 - acc: 0.4143 - val_loss: 1.2822 - val_acc: 0.3500
Epoch 9961/10000
280/280 [==============================] - 0s 105us/sample - loss: 1.0264 - acc: 0.4179 - val_loss: 1.1963 - val_acc: 0.3750
Epoch 9962/10000
280/280 [==============================] - 0s 103us/sample - loss: 1.0220 - acc: 0.4286 - val_loss: 1.3953 - val_acc: 0.3417
Epoch 9963/10000
280/280 [==============================] - 0s 96us/sample - loss: 1.0306 - acc: 0.4179 - val_loss: 1.1868 - val_acc: 0.3750
Epoch 9964/10000
280/280 [==============================] - 0s 96us/sample - loss: 1.0275 - acc: 0.4250 - val_loss: 1.2027 - val_acc: 0.3583
Epoch 99

In [15]:
# Predict all Validation data
predict = model.predict(val_x)

In [16]:
# Visualize Prediction
df = pd.DataFrame(predict)
df.columns = ['Type 0', 'Type 1', 'Type 2', 'Type 3', 'Type 4']
df.index = val_data[:,0]
print(df)

             Type 0    Type 1    Type 2    Type 3    Type 4
Enemy 281  0.516130  0.095142  0.353307  0.008790  0.026631
Enemy 282  0.253887  0.004547  0.008961  0.291646  0.440960
Enemy 283  0.606234  0.078914  0.243461  0.021751  0.049641
Enemy 284  0.432066  0.112801  0.438206  0.004182  0.012745
Enemy 285  0.253887  0.004547  0.008961  0.291646  0.440960
...             ...       ...       ...       ...       ...
Enemy 396  0.253887  0.004547  0.008961  0.291646  0.440960
Enemy 397  0.432066  0.112801  0.438206  0.004182  0.012745
Enemy 398  0.105650  0.713956  0.172374  0.003747  0.004273
Enemy 399  0.253887  0.004547  0.008961  0.291646  0.440960
Enemy 400  0.253887  0.004547  0.008961  0.291646  0.440960

[120 rows x 5 columns]
